## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config2 import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [2]:
# 1. Import the WeatherPy_database.csv file.
weather_database = "../Weather_Database/WeatherPy_Database.csv"
city_data_df = pd.read_csv(weather_database)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tasiilaq,GL,65.6145,-37.6368,30.36,97,100,10.87,snow
1,1,Punta Arenas,CL,-53.1500,-70.9167,44.71,53,75,24.16,light rain
2,2,Ushuaia,AR,-54.8000,-68.3000,38.86,70,40,23.02,light rain
3,3,Portland,US,45.5234,-122.6762,55.51,81,90,3.00,overcast clouds
4,4,Yatsushiro,JP,32.5000,130.6000,57.60,96,100,5.66,moderate rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 70
What is the maximum temperature you would like for your vacation? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Chuy,UY,-33.6971,-53.4616,73.76,67,6,9.37,clear sky
6,6,Lompoc,US,34.6391,-120.4579,84.78,38,1,12.66,clear sky
8,8,Rikitea,PF,-23.1203,-134.9692,73.71,65,0,6.13,clear sky
11,11,Meulaboh,ID,4.1363,96.1285,75.25,89,100,2.39,overcast clouds
19,19,Mahebourg,MU,-20.4081,57.7000,73.69,88,40,3.44,light rain
20,20,Vaini,TO,-21.2000,-175.2000,86.16,79,20,5.75,few clouds
21,21,Saint-Philippe,RE,-21.3585,55.7679,72.99,76,4,6.73,clear sky
22,22,Victoria,HK,22.2855,114.1577,75.25,90,44,5.01,scattered clouds
27,27,Alice Springs,AU,-23.7000,133.8833,71.15,37,31,5.75,scattered clouds
29,29,Pringsewu,ID,-5.3582,104.9744,73.80,95,100,2.44,moderate rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Chuy,UY,73.76,clear sky,-33.6971,-53.4616,
6,Lompoc,US,84.78,clear sky,34.6391,-120.4579,
8,Rikitea,PF,73.71,clear sky,-23.1203,-134.9692,
11,Meulaboh,ID,75.25,overcast clouds,4.1363,96.1285,
19,Mahebourg,MU,73.69,light rain,-20.4081,57.7000,
20,Vaini,TO,86.16,few clouds,-21.2000,-175.2000,
21,Saint-Philippe,RE,72.99,clear sky,-21.3585,55.7679,
22,Victoria,HK,75.25,scattered clouds,22.2855,114.1577,
27,Alice Springs,AU,71.15,scattered clouds,-23.7000,133.8833,
29,Pringsewu,ID,73.80,moderate rain,-5.3582,104.9744,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json() 
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [28]:
hotel_df.notnull().sum()

City                   265
Country                265
Max Temp               265
Current Description    265
Lat                    265
Lng                    265
Hotel Name             265
dtype: int64

In [19]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Chuy,UY,73.76,clear sky,-33.6971,-53.4616,Nuevo Hotel Plaza
6,Lompoc,US,84.78,clear sky,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
8,Rikitea,PF,73.71,clear sky,-23.1203,-134.9692,People ThankYou
11,Meulaboh,ID,75.25,overcast clouds,4.1363,96.1285,Kost Pria
19,Mahebourg,MU,73.69,light rain,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
20,Vaini,TO,86.16,few clouds,-21.2000,-175.2000,Keleti Beach Resort
21,Saint-Philippe,RE,72.99,clear sky,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
22,Victoria,HK,75.25,scattered clouds,22.2855,114.1577,Mini Hotel Central
27,Alice Springs,AU,71.15,scattered clouds,-23.7000,133.8833,Desert Palms Alice Springs
29,Pringsewu,ID,73.80,moderate rain,-5.3582,104.9744,Urban Style by Front One


In [22]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [33]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd> and {Max Temp} °F
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [34]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))